In [1]:
from ultralytics import YOLO
import torch
import os
from torch.utils.data import DataLoader
from ultralytics.data.dataset import YOLODataset

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/teixeira/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
model = YOLO("yolo11n.pt")  # nano version for faster prototyping

In [3]:
data_yaml = os.path.join(os.getcwd(), "../../dataset", "data.yaml")

In [ ]:
results = model.train(
    data=data_yaml,
    epochs=50,
    imgsz=640,
    batch=4,
    device=0 if torch.cuda.is_available() else 'cpu',
    augment=False,
    save=True,
    workers=4,
    name="filled_circles_model",
    verbose=False # Set verbose to False
)

# Testar o modelo
metrics = model.val()

# Acessar as métricas de precisão
precision = metrics.results_dict.get('metrics/precision(B)')
recall = metrics.results_dict.get('metrics/recall(B)')
map50 = metrics.results_dict.get('metrics/mAP50(B)')
map50_95 = metrics.results_dict.get('metrics/mAP50-95(B)')

# Imprimir as métricas de precisão no final
print("\n--- Resultados do Treinamento ---")
print(f"Precisão: {precision}")
print(f"Recall: {recall}")
print(f"mAP@50: {map50}")
print(f"mAP@50-95: {map50_95}")


In [9]:
import os
import shutil

# Caminho para o melhor modelo com base na saída
model_dir = "/home/teixeira/code/studybuddy/stb-answer-key/runs/detect/filled_circles_model/"
best_model_path = os.path.join(model_dir, "weights", "best.pt")

# Verificar e copiar
if os.path.exists(best_model_path):
    shutil.copy(best_model_path, "best.pt")
    print(f"Melhor modelo copiado como 'best.pt'")
else:
    # Alternativa: usar o último modelo
    last_model_path = os.path.join(model_dir, "weights", "last.pt")
    if os.path.exists(last_model_path):
        shutil.copy(last_model_path, "best.pt")
        print(f"Último modelo copiado como 'best.pt'")
    else:
        print(f"Nenhum modelo encontrado em {model_dir}/weights/")

Melhor modelo copiado como 'best.pt'


In [13]:
import cv2
import os
from datetime import datetime

from ultralytics import YOLO

model = YOLO("best.pt")
image_path = "/home/teixeira/code/studybuddy/stb-answer-key/images/example1.jpeg"

# Extrair nome do arquivo e extensão
filename, extension = os.path.splitext(os.path.basename(image_path))
# output_path = f"{filename}_predictions{extension}"
current_time = datetime.now().strftime("%d-%m_%H-%M-%S")  # dia-mes_hora-minuto-segundo
output_path = f"{filename}_predictions_{current_time}{extension}"

results = model(image_path)

# Obter informações das caixas
boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
confidences = results[0].boxes.conf.cpu().numpy()
class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
names = results[0].names

# Carregar imagem original
image = cv2.imread(image_path)

# Desenhar caixas
for i, box in enumerate(boxes):
    x1, y1, x2, y2 = box
    confidence = confidences[i]
    class_id = class_ids[i]
    class_name = names[class_id]
    label = f"{class_name}: {confidence:.2f}"

    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Salvar imagem com predições
cv2.imwrite(output_path, image)
print(f"Imagem salva como: {output_path}")


image 1/1 /home/teixeira/code/studybuddy/stb-answer-key/images/example1.jpeg: 640x384 24 filled_circles, 10.5ms
Speed: 1.5ms preprocess, 10.5ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)
Imagem salva como: example1_predictions_17-08_00-34-05.jpeg


In [2]:
import cv2
import os
import numpy as np
import json
from datetime import datetime
from ultralytics import YOLO

# Carrega o modelo
model = YOLO("best.pt")

# Caminho da imagem
image_path = "test_image_2.jpeg"
filename, extension = os.path.splitext(os.path.basename(image_path))
current_time = datetime.now().strftime("%d-%m_%H-%M-%S")
output_path = f"{filename}_predictions_{current_time}{extension}"

# Roda a predição
results = model(image_path)

# Acessa o primeiro resultado (suporta batch)
r = results[0]

# Extrai as informações relevantes
result_data = {
    "image_path": image_path,
    "predictions": []
}

# Loop pelas caixas detectadas
for box in r.boxes:
    prediction = {
        "class_id": int(box.cls[0]),
        "confidence": float(box.conf[0]),
        "bbox_xyxy": [float(x) for x in box.xyxy[0].tolist()]  # [x1, y1, x2, y2]
    }
    result_data["predictions"].append(prediction)

# Salva em JSON
with open("result.json", "w", encoding="utf-8") as f:
    json.dump(result_data, f, indent=4, ensure_ascii=False)

print("JSON salvo em result.json")



image 1/1 /home/teixeira/code/studybuddy/stb-cv-black-filled-circle-detector/test_image_2.jpeg: 640x384 10 filled_circles, 54.4ms
Speed: 2.9ms preprocess, 54.4ms inference, 38.1ms postprocess per image at shape (1, 3, 640, 384)
JSON salvo em result.json
